In [0]:

from pyspark.sql import functions as F

df = spark.read.jdbc("jdbc:postgresql://10.2.26.136:6432/dna", "dna_genre_id_product_mapping", properties={"user": "app_bdp_usage_qa", "password": "2mHdFW6%#REu"})
df.createOrReplaceTempView("dna_genre_id_product_mapping_136")
db_df = spark.sql("SELECT * FROM dna_genre_id_product_mapping_136")
db_df.show(10)
df1 = db_df.select(["product_id", "genre_id"]).withColumn('genre_id', F.explode('genre_id')).filter("genre_id=1")
print df1.count()


In [0]:

from pyspark.sql import functions as F

df = spark.read.jdbc("jdbc:postgresql://10.2.25.198:6432/dna", "dna_genre_id_product_mapping", properties={"user": "app_bdp_usage_qa", "password": "2mHdFW6%#REu"})
df.createOrReplaceTempView("dna_genre_id_product_mapping")
db_df = spark.sql("SELECT * FROM dna_genre_id_product_mapping")
db_df.show(10)
df1 = db_df.select(["product_id", "genre_id"]).withColumn('genre_id', F.explode('genre_id')).filter("genre_id=1")
print df1.count()




In [0]:

from pyspark.sql import functions as F

unified_db = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/_partition_col=0/")
df2 = unified_db.select(["product_id", "genre_id"]).withColumn('genre_id', F.explode('genre_id')).filter("genre_id=1")
print df2.count()




In [0]:


result = df1.union(df2).subtract(df1.intersect(df2))
print result.count()
result.show(500)



In [0]:
%%sh

# aws s3 ls "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity=daily/date=2020-01-18/"
aws s3 ls "s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/_partition_col=0/"





In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 

select date,count(distinct(genre_id)) from plproxy.execute_select_nestloop(\$proxy\$ 
    select date,genre_id
    from aa.genre_store_daily_estimate
    where 
        date between '2019-01-01' and '2019-12-31'
        
    group by date, genre_id
\$proxy\$) tbl ( date DATE, genre_id BIGINT) group by date;

EOF


In [0]:
%%sh
